<a href="https://colab.research.google.com/github/delmohan/Task/blob/main/recommenddashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask-ngrok pandas scikit-learn

import pandas as pd
from flask import Flask, request, jsonify
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from flask_ngrok import run_with_ngrok

# Sample dataset
data = {
    "product_id": [1,2,3,4,5,6,7,8,9,10],
    "product_name": ["Rice","Wheat","Shampoo","Soap","Oil","Toothpaste","Mobile","Headphones","Laptop","Charger"],
    "category": ["Groceries","Groceries","PersonalCare","PersonalCare","Groceries","PersonalCare","Electronics","Electronics","Electronics","Electronics"],
    "price": [40,35,120,30,150,50,10000,2000,50000,500]
}
df = pd.DataFrame(data)

# Feature engineering
df["features"] = df["category"] + " " + pd.cut(df["price"], bins=[0, 100, 500, 1000, 100000],
                                               labels=["Low", "Medium", "High", "Premium"])

vectorizer = TfidfVectorizer()
feature_matrix = vectorizer.fit_transform(df["features"])
similarity_matrix = cosine_similarity(feature_matrix)

# Recommendation function
def recommend_products(product_name, top_n=3):
    try:
        idx = df[df["product_name"].str.lower() == product_name.lower()].index[0]
    except IndexError:
        return []
    scores = list(enumerate(similarity_matrix[idx]))
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    recommended = [df.iloc[i[0]]["product_name"] for i in sorted_scores]
    return recommended

# Flask app
app = Flask(__name__)
run_with_ngrok(app)

@app.route("/recommend", methods=["GET"])
def recommend():
    product_name = request.args.get("product")
    if not product_name:
        return jsonify({"error": "Please provide a product name"}), 400
    recommendations = recommend_products(product_name)
    return jsonify({"product": product_name, "recommendations": recommendations})

app.run()


In [ ]:
# install only once in Colab
!pip install flask-ngrok pandas scikit-learn

import pandas as pd
from flask import Flask, request, jsonify
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from flask_ngrok import run_with_ngrok

# Sample dataset
data = {
    "product_id": [1,2,3,4,5,6,7,8,9,10],
    "product_name": ["Rice","Wheat","Shampoo","Soap","Oil","Toothpaste","Mobile","Headphones","Laptop","Charger"],
    "category": ["Groceries","Groceries","PersonalCare","PersonalCare","Groceries","PersonalCare","Electronics","Electronics","Electronics","Electronics"],
    "price": [40,35,120,30,150,50,10000,2000,50000,500]
}
df = pd.DataFrame(data)

# --- FIX: create price bucket and convert to string before concatenation ---
df["price_bucket"] = pd.cut(
    df["price"],
    bins=[0, 100, 500, 1000, 100000],
    labels=["Low", "Medium", "High", "Premium"]
)

# Option A: use str.cat and cast price_bucket to string
df["features"] = df["category"].str.cat(df["price_bucket"].astype(str), sep=" ")

# Option B (equivalent): df["features"] = df["category"] + " " + df["price_bucket"].astype(str)

# quick check
print(df[["product_name","category","price","price_bucket","features"]])

# Build TF-IDF + similarity
vectorizer = TfidfVectorizer()
feature_matrix = vectorizer.fit_transform(df["features"])
similarity_matrix = cosine_similarity(feature_matrix)

def recommend_products(product_name, top_n=3):
    try:
        idx = df[df["product_name"].str.lower() == product_name.lower()].index[0]
    except IndexError:
        return []
    scores = list(enumerate(similarity_matrix[idx]))
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    return [df.iloc[i[0]]["product_name"] for i in sorted_scores]

# Flask app (Colab + ngrok)
app = Flask(__name__)
run_with_ngrok(app)

@app.route("/recommend", methods=["GET"])
def recommend():
    product_name = request.args.get("product")
    if not product_name:
        return jsonify({"error": "Please provide a product name"}), 400
    return jsonify({"product": product_name, "recommendations": recommend_products(product_name)})

app.run()


  product_name      category  price price_bucket             features
0         Rice     Groceries     40          Low        Groceries Low
1        Wheat     Groceries     35          Low        Groceries Low
2      Shampoo  PersonalCare    120       Medium  PersonalCare Medium
3         Soap  PersonalCare     30          Low     PersonalCare Low
4          Oil     Groceries    150       Medium     Groceries Medium
5   Toothpaste  PersonalCare     50          Low     PersonalCare Low
6       Mobile   Electronics  10000      Premium  Electronics Premium
7   Headphones   Electronics   2000      Premium  Electronics Premium
8       Laptop   Electronics  50000      Premium  Electronics Premium
9      Charger   Electronics    500       Medium   Electronics Medium
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-3:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.12/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
            